In [3]:
from collections import Counter
from datetime import datetime
import requests
import time

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select

from IPython.display import Audio
sound_file = './goes-without-saying-608.mp3'

In [13]:
url = "https://www.sante.fr/cf/centres-vaccination-covid.html"
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')


nodoctolib_list = []
doctolib_list=[]
maiia_list=[]
keldoc_list=[]
list_dep = ["78","92","75"] #'91',"95","93","94","28","27"]
    

for dep in list_dep: 
    for link in soup.find("li", {"id":"dep-"+dep}).find_all('a'):
        link = link.get('href')      
        if link.startswith('https://partners.doctolib.fr') and "professionnel" not in link  :
            doctolib_list.append(link)
        elif link.startswith('https://www.maiia.com/'):
            maiia_list.append(link)
        elif link.startswith('https://vaccination-covid.keldoc.com/'):
            keldoc_list.append(link)
        else:   
            pass
        
doctolib_list.append('https://partners.doctolib.fr/centre-de-vaccinations-internationales/longjumeau/centre-de-vaccination-covid-19-ville-de-longjumeau?speciality_id=5494&enable_cookies_consent=1')

In [14]:
#Remove cookies
doctolib_list = [link[:-1]+'0' for link in doctolib_list]

In [5]:
print(len(doctolib_list))
print(len(maiia_list))
print(len(keldoc_list))

39
1
0


In [16]:
dict_list = {}
for i in range(len(doctolib_list)):
    centre_type = doctolib_list[i].split('/')[3]
    if centre_type in dict_list:
        dict_list[centre_type]+=1
    else:
        dict_list[centre_type]=1
        
for k, v in sorted(dict_list.items()): print(k, v)

centre-de-sante 30
centre-de-vaccinations-internationales 8
centre-depistage-covid 1


In [17]:
#create first column dataframe
centre_list=[]
for url in doctolib_list:
    centre_vaccination = url.split("?")[0].split("/")[-1]
    if centre_vaccination.startswith("centre-de-vaccination-covid-19-de-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-19-de-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-covid-de-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-de-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-covid-19-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-19-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-covid-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-")[1]
    elif centre_vaccination.startswith("ch-eure-seine-centre-de-vaccination-covid-"):
        centre_vaccination=centre_vaccination.split("ch-eure-seine-centre-de-vaccination-covid-")[1]
        
    centre_list.append(centre_vaccination)
df = pd.DataFrame(centre_list, columns=['centre'])

In [ ]:

## TO LAUNCH THE VERY FIRST TIME ONLY, use the next cell after that
## perhaps 'use the wait until presence of element located method' instead of sleep
options = webdriver.ChromeOptions()
#options.add_argument('headless')

driver = webdriver.Chrome(chrome_options=options)
wait_sec=0.5

for trial in range(1,30):
    number_vaccination_list=[]
    list_with_issues=[]
    
    for i,url in enumerate(doctolib_list):

        try:
            driver.get(url)
            time.sleep(wait_sec)
            select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
            select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Pfizer-BioNTech)')
            time.sleep(wait_sec)
            driver.find_element_by_xpath("//*[@id='booking-content']/div/div[3]/div[2]/div/div[1]/div/div/div[2]/div/div/div/button").click()
            Audio(sound_file,rate = 22050, autoplay=True)
            print(url)
            break
        except NoSuchElementException:

            try:        

                select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
                select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Pfizer-BioNTech)')
                time.sleep(wait_sec)
                answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                if answer == ['Vous', 'ne', 'pouvez', 'pas', 'prendre', 'rendez-vous', 'en', 'ligne', 'avec', 'ce', 'praticien.']:
                    num_vaccinations = '9999'
                else:
                    num_vaccinations=answer[12]
                number_vaccination_list.append(num_vaccinations)
                print("n."+ str(i), centre_list[i] ,num_vaccinations)
            except NoSuchElementException:  

                try:
                    answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                    if answer == ['Désolé,', 'une', 'erreur', 'est', 'survenue.']:
                        num_vaccinations = '9999'
                    if answer == ['Vous', 'ne', 'pouvez', 'pas', 'prendre', 'rendez-vous', 'en', 'ligne', 'avec', 'ce', 'praticien.']:
                        num_vaccinations = '9999'
                    else:
                        num_vaccinations =answer[12]
                    number_vaccination_list.append(num_vaccinations)
                    print("n."+ str(i), centre_list[i] ,num_vaccinations)
                except (IndexError, NoSuchElementException) :

                    try:
                        driver.find_element_by_xpath("//*[@id='booking-content']/div/div[1]/div[3]/label[2]").click()
                        select_fr = Select(driver.find_element_by_xpath("//*[@id='booking_motive']"))
                        select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Pfizer-BioNTech)')
                        time.sleep(wait_sec)
                        answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                        num_vaccinations=answer[12]
                        number_vaccination_list.append(num_vaccinations)
                        print("n."+ str(i), centre_list[i] ,num_vaccinations)

                    except (IndexError, NoSuchElementException) :
                       
                        try:
                            select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
                            select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Moderna)')
                            time.sleep(wait_sec)
                            driver.find_element_by_xpath("//*[@id='booking-content']/div/div[3]/div[2]/div/div[1]/div/div/div[2]/div/div/div/button").click()
                            Audio(sound_file,rate = 22050, autoplay=True)
                            print(url)
                        except (IndexError, NoSuchElementException) :
                            
                            try:
                                select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
                                select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Moderna)')
                                time.sleep(wait_sec)
                                answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                                if answer == ['Vous', 'ne', 'pouvez', 'pas', 'prendre', 'rendez-vous', 'en', 'ligne', 'avec', 'ce', 'praticien.']:
                                    num_vaccinations = '9999'
                                else:
                                    num_vaccinations=answer[12]
                                number_vaccination_list.append(num_vaccinations)
                                print("n."+ str(i), centre_list[i] ,num_vaccinations)
                            
                            except (IndexError, NoSuchElementException) : 
                            
                                number_vaccination_list.append("0")
                                list_with_issues.append(url)

                                pass


    now = datetime.now()
    current_time = now.strftime("%m/%d-%H:%M:%S")
    values_df = pd.DataFrame(number_vaccination_list, columns= [current_time])
    values_df[current_time]=pd.to_numeric(values_df[current_time].replace('tous','9999'))
    df = df.join(values_df)

    pd.set_option("display.max_rows", None, "display.max_columns", None)
    if len(df.columns) >2 :
        display(df[df.iloc[:,-2]>df.iloc[:,-1]].sort_values(current_time))
    display(df)
    
    print('trial n.', str(trial), 'at', str(current_time) , 'terminé. Sleep for 40 min before next round')
    
    time.sleep(1200)

    

/home/yamhiroto/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


n.0 houdan 632
n.1 cpts-val-de-seine 1589
n.2 montigny-le-bretonneux 945
n.3 rambouillet 814
n.4 saint-germain-en-laye 1791
n.5 de-saint-remy-les-chevreuse 2126
n.6 versailles 500
n.7 espace-concorde-francis-delage 730
n.8 centre-vaccination-ville-de-boulogne-billancourt 2155
n.9 de-chatenay-malabry 9999
n.10 la-ville-de-clamart 855
n.11 clichy 1533
n.12 courbevoie 1690
n.13 issy-les-moulineaux 1900
n.14 la-garenne-colombes 1554
n.15 du-theatre-des-sablons tous
n.16 europe-rueil-malmaison 1845
n.17 suresnes 1983
n.19 centre-covid19-paris-5 1381
n.20 mairie-du-6eme-arrondissement-de-paris 66
n.21 mairie-du-7eme-paris 2050
n.23 mairie-du-9eme-arrondissement 772
n.24 centre-medical-international-cmi 1055
n.26 centre-de-sante-bauchat-nation 2091
n.27 ville-de-paris 552
n.29 paris-14e 530
n.30 paris-15e 1067
n.31 centre-medical-institut-pasteur-cmip-vaccination-covid-19 2395
n.32 centre-henry-dunant-croix-rouge-francaise 1787
n.33 sos-medecins-paris 2033
n.34 ars-idf-centre-covisan-cpts-par

,centre,03/01-22:52:12
0,houdan,632
1,cpts-val-de-seine,1589
2,montigny-le-bretonneux,945
3,rambouillet,814
4,saint-germain-en-laye,1791
5,de-saint-remy-les-chevreuse,2126
6,versailles,500
7,espace-concorde-francis-delage,730
8,centre-vaccination-ville-de-boulogne-billancourt,2155
9,de-chatenay-malabry,9999


trial n. 1 at 03/01-22:52:12 terminé. Sleep for 40 min before next round


# List with Issues

In [ ]:
list_with_issues

In [ ]:
### TEST WITH WEBDRIVERWAIT RATHER THAN SLEEP 

options = webdriver.ChromeOptions()
#options.add_argument('headless')
driver = webdriver.Chrome(chrome_options=options)

for i,url in enumerate(doctolib_list):
    try:        
        driver.get(url)
        element_present = EC.presence_of_element_located((By.ID, 'booking_motive'))
        WebDriverWait(driver, 2).until(element_present)
        select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
        select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Pfizer-BioNTech)')
        element_present2 = EC.presence_of_element_located((By.CLASS_NAME, 'dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker'))
        WebDriverWait(driver, 2).until(element_present2)
        answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
        num_vaccinations=answer[12]
        number_vaccination_list.append(num_vaccinations)
        print("n."+ str(i), centre_list[i] ,num_vaccinations)
    except (NoSuchElementException, TimeoutException):  
        
        try:
            driver.get(url)
            element_present = EC.presence_of_element_located((By.ID, 'booking_motive'))
            WebDriverWait(driver, 2).until(element_present)
            answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
            num_vaccinations =answer[12]
            number_vaccination_list.append(num_vaccinations)
            print("n."+ str(i), centre_list[i] ,num_vaccinations)
        except (IndexError, NoSuchElementException, TimeoutException) :
            number_vaccination_list.append("0")
            list_with_issues.append(url)
            #doctolib_list.remove(url)
            pass
    